###Prepare the connection
1. Apply a Google Places Web Service API key on [Google Developers Console](http://console.developers.google.com/)
2. Save the key as an json file in the same folder of the python script
3. import the modules we need and set up a credential connection to Google Places API
(If you didn't install 'googleplaces' module before, go to terminal and type `pip install https://github.com/slimkrazy/python-google-places/zipball/master` to install the module)

In [7]:
import json  #for reading oauth info and save the results
import io 
from googleplaces import GooglePlaces, types, lang
from pprint import pprint

In [1]:
with io.open('google_places_key.json') as cred:
    creds = json.load(cred)
    google_places = GooglePlaces(**creds)

### Get data from API
Every account can excuete 1,000 API calls within 24 hours. The response limit of every search is 20. So again we have to narrow down the search criteria to get more data. Let's begin with the geometry of Boston Downtown Crossing.
> For the parameters you can set up in the following query, go to [Documentation](https://developers.google.com/places/web-service/search)

Then we check if we get any results from API and print them to the screen. We use pprint module to let the results display in a readable format:

In [20]:
    if query_result.raw_response:
        pprint(query_result.raw_response)

{u'html_attributions': [],
 u'next_page_token': u'CoQC-AAAAIVPj-smopYp9L9m4jakOqAVmryKUZp9MB8iW8TtlwIbSJpRjeezMGknv0y8dbxDHry4z29J1ML1RmXUXMBVLZxY7nMXSLesql7R3wOU5jrZFR7_B1Zo4vOM6RNCc79LPlgsgIlWCiqXgA-VWmeniiQzVI5x3TN2NmkqmC8UkqXEJqzM0scm1Ts4wWTiD0BcYvM39zv0eUJql3Ub8Emb8wFnKVY8kl_VInBPMqjJULyD7dOLn8eCC7MOnfdfwbjaSDxk-v8F867md4jkMHr9bQXaedsW3E4F4AkTn6vau3GKWYlvqcirusGyo3cGMn2ZXlA6vnwvjbhvn5O-llzZSYwSEMHJYhuj65PLpIct1ds18OwaFNebw-l22fnuukZ43DF24PPNVOoh',
 u'results': [{u'geometry': {u'location': {u'lat': Decimal('42.3552522'),
                                           u'lng': Decimal('-71.0631123')}},
               u'icon': u'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
               u'id': u'be4e4ce5b3f8c5e7dcc2c062f513b3fd6b80ac8b',
               u'name': u"McDonald's",
               u'opening_hours': {u'open_now': True, u'weekday_text': []},
               u'photos': [{u'height': 800,
                            u'html_attributions': [u'<a href="https://m

The response from API above contains many information:
1. A 'next_page_token': Use this token at your next search assigned to parameter 'pagetoken' and you'll get the following 20 results of your previous serach.
2. Your 'results' of places: These are the data we want. But only few basic informatino here, so we have to get more details in the follwing steps.

### Look into details of the data
We use `vars(object)` to extend all the information in the object:
> To see all the response values and their definitinos in places, go to [Google Places Search Results](https://developers.google.com/places/web-service/search#PlaceSearchResults)

In [18]:
    for place in query_result.places:
        pprint(vars(place))  #only get geo_location, icon, id, name, place_id, rating, types, vicinty 
        # The following method has to make a further API call.
        place.get_details()  #get more details including phone_number, opening_hours, photos, reviews ... etc
        pprint(vars(place))

{'_details': None,
 '_geo_location': {u'lat': Decimal('42.3552522'),
                   u'lng': Decimal('-71.0631123')},
 '_icon': u'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
 '_id': u'be4e4ce5b3f8c5e7dcc2c062f513b3fd6b80ac8b',
 '_name': u"McDonald's",
 '_place_id': u'ChIJe645PYNw44kR_0ilqhElTKw',
 '_query_instance': <googleplaces.GooglePlaces object at 0x1072d2c50>,
 '_rating': Decimal('2.8'),
 '_reference': u'CmReAAAAbphiaGMy7Pq_lOYSigfosfCrxGdsYXRRQ1tAKf9qzLWYn9OqlubHPb9Kvz0e8MS2umyQTesG3facf9R_5O1SZgRUhCSqSObhLiz07u5K5EVDWiCtdntg1SnLBQi5RJw0EhB-PCetYoAMKs3Ub6Y6TjcBGhT465IGL797_4DuNdOu33tnjmsCUQ',
 '_types': [u'restaurant', u'food', u'point_of_interest', u'establishment'],
 '_vicinity': u'146 Tremont Street, Boston'}
{'_details': {u'address_components': [{u'long_name': u'146',
                                       u'short_name': u'146',
                                       u'types': [u'street_number']},
                                      {u'long_nam

### Scrape the data nad save them to json files
Let's start to collect the data and save them to files for further usage. We list all the criterias we want to search for, go through them one by one, append to a list and save the list as a json file.

In [ ]:
    result = []
    #Put your lantitude and longtitude pairs in the list and run the search in turns
    lan_lng_list = [{'lan': 2.356357, 'lng': -71.0623345},  #Park Street Station
                    {'lan': 42.356357, 'lng': -71.0623345},  #China Town Station
                    {'lat': 42.3555885, 'lng': -71.0646816}] #Downtown Crossing Station
    for pair in lan_lng_list:
        query_result = google_places.nearby_search(
        lat_lng = pair, rankby = 'distance', types = [types.TYPE_FOOD])
        for place in query_result.places:
            place.get_details()
            tmp = vars(place)
            results.append(tmp)
            
    with open('my_boston_restaurants_google_places.json', 'wb') as f:
        results_json = json.dumps(results, indent=4, skipkeys=True, sort_keys=True)
        f.write(results_json)

### Reference
1. slimkrazy's Google Places module: [python-google-places](https://github.com/slimkrazy/python-google-places)
2. [Documentation](https://developers.google.com/places/web-service/) of Google Places